<a href="https://colab.research.google.com/github/popelucha/digital-humanities/blob/main/DH_3_Text_Analysis_Using_APIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Analysis using APIs
The aim of the notebook is to begin with quantitative analysis of text data. We select a Czech text, split it into tokens, perform frequency analysis, and observe the nature of the data.

Particularly, we use the Czech tagger `desamb` via Language Services API at Natural Language Processing Centre, Faculty of Informatics, Masaryk University Brno.

The tagset is described in https://nlp.fi.muni.cz/raslan/2011/paper05.pdf.


JAKUBÍČEK, Miloš, Vojtěch KOVÁŘ a Pavel ŠMERK. Czech Morphological Tagset Revisited. In Horák, Rychlý. Proceedings of Recent Advances in Slavonic Natural Language Processing 2011. Brno: Tribun EU, 2011. s. 29-42, 14 s. ISBN 978-80-263-0077-9. https://www.muni.cz/en/research/publications/959110

In [ ]:
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from collections import Counter
import numpy as np

In [ ]:
text = None
with open('maj.txt') as f:  # modify the path if needed
    text = f.read()

In [ ]:
import requests
import json

# Make the API call
In this step, we rely on the service provided by the 3rd party. It impacts not only the actual need of annotation but also reproducibility of the results.

The result of the API call is a HTTP code (since the API is called via HTTP POST request). You can find the meaning of the numbers here: https://www.restapitutorial.com/httpstatuscodes.html. Usually, only few codes appear as the API call status code. For example, 200 means OK, the 404 is well known Not found.

In [ ]:
data = {"call": "tagger",
        "lang": "cs",
        "output": "json",
        "text": text.replace(';', ',')
       }
uri = "https://nlp.fi.muni.cz/languageservices/service.py"
r = requests.post(uri, data=data)
r

In [ ]:
print(r.content[:1000].decode('utf-8'))

Note that we convert automatically the data into JSON. JSON is one of the communication standards that is used when two machines communicate.

In [ ]:
data = r.json()
data

**TASK 1**: Observe the data. Without the tagger documentation, is it possible to understand the data?

Explanations: The tagger splits text into sentences (delimited by `<s>` and `</s>`), then into tokens. The tokens are usually separated by spaces. In case they are not (typically punctuation), the tagger adds the "glue" tag `<g/>`. Each token is represented by a list with three elements: word, lemma, tag.

**TASK 2** Check the paper describing the tagset and try to "decipher" annotation for one token into human readable annotation such as "feminine noun in plural instrumental".

In [ ]:
tokens = [token for token in data['vertical'] if len(token)==3]
df = pd.DataFrame.from_dict({"word": [word for word, lemma, tag in tokens],
                              "lemma": [lemma for word, lemma, tag in tokens],
                              "tag": [tag for word, lemma, tag in tokens]
                               })
pd.options.display.max_rows = len(df)
df

In [ ]:
pos = [tag[0:2] for tag in df["tag"]]
df["pos"] = pos
df